In [10]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter('ignore')

In [11]:
data = pd.read_csv(r'C:/Users/uzmap/Documents/GitHub/ForEx/data_For_prophet.csv',index_col='Unnamed: 0')
data.head()

,Unnamed: 0.1,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Trend_Open_price,Seasonal_Open_price,Residual_Open_price,...,Seasonal_Day_low,Residual_Day_low,RSI,MACD,MACD_Signal,BB_Upper,BB_Lower,day_of_week,month,is_weekend
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0.000000,0,2014-11-07,0.386891,0.384657,0.389693,0.386954,USD/INR,0.394461,0.522182,0.60751,...,0.473615,0.538767,64.491363,0.000000,0.000000,0.391812,0.38654,4,11,0
0.000055,1,2014-11-10,0.387590,0.384752,0.389693,0.387558,USD/INR,0.394461,0.557718,0.60751,...,0.460915,0.538767,64.491363,0.000048,0.000010,0.391812,0.38654,0,11,0
0.000109,2,2014-11-11,0.387781,0.384248,0.390750,0.387641,USD/INR,0.394461,0.533733,0.60751,...,0.475153,0.538767,64.491363,0.000092,0.000026,0.391812,0.38654,1,11,0
0.000164,3,2014-11-12,0.387641,0.384280,0.389757,0.386897,USD/INR,0.394461,0.541190,0.60751,...,0.422273,0.538767,64.491363,0.000066,0.000034,0.391812,0.38654,2,11,0
0.000219,4,2014-11-13,0.386752,0.384676,0.389757,0.388003,USD/INR,0.394461,0.534029,0.60751,...,0.456504,0.538767,64.491363,0.000133,0.000054,0.391812,0.38654,3,11,0


In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

# Convert Date column to datetime (if necessary)
data['Date'] = pd.to_datetime(data['Date'])

# List of unique currency pairs
currency_pairs = data['Currency Pair'].unique()

# Dictionary to store evaluation metrics for each currency pair
results = {}

# Loop over each currency pair
for pair in currency_pairs:
    print(f"Training model for currency pair: {pair}")
    
    # Filter data for the current currency pair
    pair_data = data[data['Currency Pair'] == pair]
    
    # Sort by date to ensure proper time series order
    pair_data = pair_data.sort_values('Date')
    
    # Prepare data for Prophet (needs columns 'ds' and 'y')
    df_prophet = pair_data[['Date', 'Closing_price']].rename(columns={'Date': 'ds', 'Closing_price': 'y'})
    
    # Add the regressors to the data (these are assumed to be already in the dataset)
    regressors = [
        'RSI', 'MACD', 'MACD_Signal', 'BB_Upper', 'BB_Lower', 
        'is_weekend','Seasonal_Open_price'
    ]
    
    for regressor in regressors:
        df_prophet[regressor] = pair_data[regressor]
    
    # Split data into train and test sets (80-20 split)
    split_idx = int(len(df_prophet) * 0.8)
    train_data = df_prophet.iloc[:split_idx]
    test_data = df_prophet.iloc[split_idx:]
    
    # Initialize the Prophet model and add regressors
    model = Prophet(daily_seasonality=True, yearly_seasonality=True, weekly_seasonality=True)
    
    # Add each regressor to the model
    for regressor in regressors:
        model.add_regressor(regressor)
    
    # Fit the model
    model.fit(train_data)

    # Make future DataFrame for predictions (including the regressors)
    future = model.make_future_dataframe(periods=len(test_data), freq='D')
    
    # Add the regressors to the future DataFrame
    for regressor in regressors:
        future[regressor] = pair_data[regressor].iloc[-len(future):].values
    
    # Forecast using Prophet
    forecast = model.predict(future)
    
    # Extract the forecasted values for the test period
    y_pred_test = forecast['yhat'].iloc[split_idx:].values
    y_test = test_data['y'].values

    # Calculate evaluation metrics
    mae = mean_absolute_error(y_test, y_pred_test)
    mape = mean_absolute_percentage_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    
    # Store the results and predictions for this currency pair
    results[pair] = {
        'MAE': mae,
        'MAPE': mape,
        'R^2': r2,
        'y_true': y_test,       # True values for the test set
        'y_pred': y_pred_test    # Predicted values for the test set
    }
    
    print(f"Results for {pair} - MAE: {mae}, MAPE: {mape}, R^2: {r2}")

# Display final results and predictions for each currency pair
print("\nFinal Results for Each Currency Pair:")
for pair, metrics in results.items():
    print(f"{pair} - MAE: {metrics['MAE']}, MAPE: {metrics['MAPE']}, R^2: {metrics['R^2']}")
    print(f"True values (sample): {metrics['y_true'][:5]}")
    print(f"Predicted values (sample): {metrics['y_pred'][:5]}\n")


Training model for currency pair: USD/INR


15:02:06 - cmdstanpy - INFO - Chain [1] start processing
15:02:08 - cmdstanpy - INFO - Chain [1] done processing
15:02:08 - cmdstanpy - INFO - Chain [1] start processing


Results for USD/INR - MAE: 0.013581753661152496, MAPE: 0.025940130904792285, R^2: -27.54595298745711
Training model for currency pair: EUR/USD


15:02:09 - cmdstanpy - INFO - Chain [1] done processing


Results for EUR/USD - MAE: 0.001086689193030175, MAPE: 0.29804992882343706, R^2: -262.12791953944634
Training model for currency pair: GBP/USD


...

Results for EUR/INR - MAE: 0.03563335973100193, MAPE: 0.06287377430774448, R^2: -31.21905077514303
Training model for currency pair: JPY/INR


15:02:14 - cmdstanpy - INFO - Chain [1] done processing
15:02:14 - cmdstanpy - INFO - Chain [1] start processing


Results for JPY/INR - MAE: 1.4082165481144417e-05, MAPE: 0.03725310956548428, R^2: 0.9892093222964288
Training model for currency pair: GBP/INR


15:02:15 - cmdstanpy - INFO - Chain [1] done processing


Results for GBP/INR - MAE: 0.012344569957368079, MAPE: 0.018762701820697135, R^2: -0.013940814588569106

Final Results for Each Currency Pair:
USD/INR - MAE: 0.013581753661152496, MAPE: 0.025940130904792285, R^2: -27.54595298745711
True values (sample): [0.51128423 0.51189428 0.51432814 0.51519238 0.51543386]
Predicted values (sample): [0.46444497 0.46498367 0.51259188 0.51358946 0.51458603]

EUR/USD - MAE: 0.001086689193030175, MAPE: 0.29804992882343706, R^2: -262.12791953944634
True values (sample): [0.00335682 0.00338039 0.00335923 0.00333966 0.00328552]
Predicted values (sample): [0.00332158 0.00335504 0.00337143 0.00046857 0.00011433]

GBP/USD - MAE: 3.2319004475678095e-05, MAPE: 0.006820049734633937, R^2: 0.9629262528833027
True values (sample): [0.00432432 0.0043465  0.00431148 0.00432813 0.00429077]
Predicted values (sample): [0.00427888 0.00431492 0.0043348  0.00431994 0.00430541]

USD/JPY - MAE: 0.006304817510808152, MAPE: 0.007325400381143641, R^2: 0.9360425531277172
True va

In [14]:
df_prophet[regressors].corr()

,RSI,MACD,MACD_Signal,BB_Upper
RSI,1.000000,0.698538,0.497529,0.027997
MACD,0.698538,1.000000,0.940020,0.134155
MACD_Signal,0.497529,0.940020,1.000000,0.193931
BB_Upper,0.027997,0.134155,0.193931,1.000000
